# Notebook05: POPC/cholesterol mixture

In this notebook, we will show you how to reconstruct hydrogens, calculate the order parameters and produce output trajectories on a POPC/cholesterol (66:34) mixture. Again, this example is based on the Berger united-atom force field.

Before going on, we advise you to get started with [Notebook01](Notebook_01_buildH_calc_OP.ipynb) if you are not familiar with basic features of **buildH** and Jupyter notebooks (e.g. do you know what `!` means in a Jupyter cell?). 

## Checking buildH activation

As explained in [Notebook01](Notebook_01_buildH_calc_OP.ipynb), you should have activated **buildH** before launching this notebook, thus you should obtain the following when invoking `buildH` in a Unix terminal:

In [1]:
!buildH

usage: buildH [-h] [-v] -c COORD [-t TRAJ] -l LIPID
              [-lt LIPID_TOPOLOGY [LIPID_TOPOLOGY ...]] -d DEFOP
              [-opx OPDBXTC] [-o OUT] [-b BEGIN] [-e END]
buildH: error: the following arguments are required: -c/--coord, -l/--lipid, -d/--defop


If you see this, you can go on. If not, please go to `Notebook01` and see the explanations there on how to activate **buildH**.

## Downloading the example files

The example shown in this Notebook are taken from a POPC/cholesterol (66:34) trajectory (84 POPC molecules and 44 cholesterol molecules) taken from the [NMRlipidsIVb project](http://nmrlipids.blogspot.com/). The initial source files can be found on [Zenodo](https://zenodo.org/record/4643985). For the purpose of learning, we simplified the trajectory and extracted only 25 frames (0-25 ns, one frame / ns). We also made the molecules whole with the tool [`trjconv`](https://manual.gromacs.org/documentation/current/onlinehelp/gmx-trjconv.html) from GROMACS with the flag `-pc mol`. The starting gro file and the trajectory of 25 frames can be directly downloaded from the **buildH** repo in the directory https://github.com/patrickfuchs/buildH/tree/master/docs/Berger_POPCCHOL_test_case. Here we'll download them with `wget` (recall to download all files in raw format):

In [2]:
!wget https://raw.githubusercontent.com/patrickfuchs/buildH/master/docs/Berger_POPCCHOL_test_case/endCONF_OK.gro

--2021-06-28 13:07:56--  https://raw.githubusercontent.com/patrickfuchs/buildH/master/docs/Berger_POPCCHOL_test_case/endCONF_OK.gro
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1795883 (1,7M) [text/plain]
Enregistre : «endCONF_OK.gro»

endCONF_OK.gro      100%[===================>]   1,71M  --.-KB/s    ds 0,04s   

2021-06-28 13:07:57 (41,4 MB/s) - «endCONF_OK.gro» enregistré [1795883/1795883]



In [3]:
!wget https://github.com/patrickfuchs/buildH/raw/master/docs/Berger_POPCCHOL_test_case/popcCHOL34molPER0-25ns_OK_dt1000.xtc

--2021-06-28 13:07:58--  https://github.com/patrickfuchs/buildH/raw/master/docs/Berger_POPCCHOL_test_case/popcCHOL34molPER0-25ns_OK_dt1000.xtc
Résolution de github.com (github.com)… 140.82.121.4
Connexion à github.com (github.com)|140.82.121.4|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://raw.githubusercontent.com/patrickfuchs/buildH/master/docs/Berger_POPCCHOL_test_case/popcCHOL34molPER0-25ns_OK_dt1000.xtc [suivant]
--2021-06-28 13:07:59--  https://raw.githubusercontent.com/patrickfuchs/buildH/master/docs/Berger_POPCCHOL_test_case/popcCHOL34molPER0-25ns_OK_dt1000.xtc
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 2472992 (2,4M) [application/octet-stream]
Enregistre : «popcCHOL34molPER0-25ns_OK_dt1

In [4]:
!ls

Berger_POPC.def
endCONF_OK.gro
img
Notebook_01_buildH_calc_OP.ipynb
Notebook_02_buildH_calc_OP_outputwH.ipynb
Notebook_03_mixture_POPC_POPE.ipynb
Notebook_04_library.ipynb
Notebook_05_mixture_POPC_cholesterol.ipynb
OP_POPC.dat
popc0-25ns_dt1000.xtc
popcCHOL34molPER0-25ns_OK_dt1000.xtc
start_128popc.pdb


We also need the two def files corresponding to POPC and cholesterol. They can be downloaded from the **buildH** repo in the directory https://github.com/patrickfuchs/buildH/tree/master/def_files (again remember to use raw files):

In [5]:
!wget https://raw.githubusercontent.com/patrickfuchs/buildH/master/def_files/Berger_POPC.def

--2021-06-28 13:08:05--  https://raw.githubusercontent.com/patrickfuchs/buildH/master/def_files/Berger_POPC.def
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 2135 (2,1K) [text/plain]
Enregistre : «Berger_POPC.def.1»

Berger_POPC.def.1   100%[===================>]   2,08K  --.-KB/s    ds 0s      

2021-06-28 13:08:05 (52,9 MB/s) - «Berger_POPC.def.1» enregistré [2135/2135]



In [6]:
!wget https://raw.githubusercontent.com/patrickfuchs/buildH/master/def_files/Berger_CHOL.def

--2021-06-28 13:08:06--  https://raw.githubusercontent.com/patrickfuchs/buildH/master/def_files/Berger_CHOL.def
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1275 (1,2K) [text/plain]
Enregistre : «Berger_CHOL.def»

Berger_CHOL.def     100%[===================>]   1,25K  --.-KB/s    ds 0s      

2021-06-28 13:08:07 (119 MB/s) - «Berger_CHOL.def» enregistré [1275/1275]



In [7]:
!ls

Berger_CHOL.def
Berger_POPC.def
Berger_POPC.def.1
endCONF_OK.gro
img
Notebook_01_buildH_calc_OP.ipynb
Notebook_02_buildH_calc_OP_outputwH.ipynb
Notebook_03_mixture_POPC_POPE.ipynb
Notebook_04_library.ipynb
Notebook_05_mixture_POPC_cholesterol.ipynb
OP_POPC.dat
popc0-25ns_dt1000.xtc
popcCHOL34molPER0-25ns_OK_dt1000.xtc
start_128popc.pdb


Recall, the def files present on the **buildH** `def_files` directory contain all the possible (apolar) H that can be reconstructed on each lipids. The polar hydrogen of cholesterol is already present, thus it does not need to be reconstructed. **buildH** will let it as is, like all other non apolar H atoms.

## Dealing with POPC

When we have a mixture of lipids, **buildH** cannot reconstruct hydrogens on all lipids at the same time. So we start with H reconstruction and OP calculation on POPC (we'll focus on cholesterol later). Since Berger POPC is a supported lipid, we can launch easily **buildH** with the classical flags: `-c` for reading the initial pdb or gro file, `-t` for reading the trajectory, `-l` for specifying the type of lipid, `-d` for specifying the def file, `-o` for the output file with order parameters, and last `-opx` for the output trajectory with hydrogens (recall not to put any extension, they'll be added automatically by **buildH**). Importantly, when we want to output a trajectory with newly reconstructed hydrogens (option `-opx`), *all possible H to reconstruct need to be present in the def file*. This is fine since this is the case for all lipids in the [def file directory](https://github.com/patrickfuchs/buildH/tree/master/def_files) of the  **buildH** repository.

In [8]:
!buildH -c endCONF_OK.gro -t popcCHOL34molPER0-25ns_OK_dt1000.xtc -l Berger_PLA -d Berger_POPC.def -o OP_POPC.out -opx traj_POPC_wH

Constructing the system...
/home/pierre/.soft/miniconda3/envs/buildh/lib/python3.9/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  residx = np.zeros_like(criteria[0], dtype=np.int)
/home/pierre/.soft/miniconda3/envs/buildh/lib/python3.9/site-packages/MDAnalysis/core/selection.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar ty

In [10]:
!ls

Berger_CHOL.def
Berger_POPC.def
Berger_POPC.def.1
endCONF_OK.gro
img
Notebook_01_buildH_calc_OP.ipynb
Notebook_02_buildH_calc_OP_outputwH.ipynb
Notebook_03_mixture_POPC_POPE.ipynb
Notebook_04_library.ipynb
Notebook_05_mixture_POPC_cholesterol.ipynb
OP_POPC.dat
OP_POPC.out
popc0-25ns_dt1000.xtc
popcCHOL34molPER0-25ns_OK_dt1000.xtc
start_128popc.pdb
traj_POPC_wH.pdb
traj_POPC_wH.xtc


The order parameters of POPC lipids have been written to the file `OP_POPC.dat`

In [11]:
!head OP_POPC.out

# OP_name            resname atom1 atom2  OP_mean OP_stddev OP_stem
#--------------------------------------------------------------------
gamma1_1             PLA     C1    H11    0.03609  0.16545  0.01805
gamma1_2             PLA     C1    H12   -0.01327  0.09031  0.00985
gamma1_3             PLA     C1    H13   -0.01714  0.11162  0.01218
gamma2_1             PLA     C2    H21    0.03573  0.16452  0.01795
gamma2_2             PLA     C2    H22   -0.01077  0.10390  0.01134
gamma2_3             PLA     C2    H23    0.00015  0.09757  0.01065
gamma3_1             PLA     C3    H31    0.03542  0.16105  0.01757
gamma3_2             PLA     C3    H32   -0.01826  0.09667  0.01055


A pdb `traj_POPC_wH.pdb` and an xtc `traj_POPC_wH.xtc` file have been produced with hydrogens on POPC only. So let's check whether the POPC have hydrogens and not cholesterol (recall, here POPC residue name is PLA):

In [12]:
!grep PLA traj_POPC_wH.pdb | head -20

ATOM      1  C1   PLA    1      10.670  42.050  27.850  1.00  0.00             C
ATOM      2  H11  PLA    1      10.727  41.123  27.280  1.00  0.00             H
ATOM      3  H12  PLA    1       9.962  41.929  28.670  1.00  0.00             H
ATOM      4  H13  PLA    1      11.654  42.290  28.252  1.00  0.00             H
ATOM      5  C2   PLA    1       8.900  42.780  26.440  1.00  0.00             C
ATOM      6  H21  PLA    1       8.966  41.826  25.916  1.00  0.00             H
ATOM      7  H22  PLA    1       8.564  43.552  25.748  1.00  0.00             H
ATOM      8  H23  PLA    1       8.189  42.694  27.262  1.00  0.00             H
ATOM      9  C3   PLA    1      11.120  43.130  25.800  1.00  0.00             C
ATOM     10  H31  PLA    1      11.123  42.136  25.353  1.00  0.00             H
ATOM     11  H32  PLA    1      12.130  43.389  26.117  1.00  0.00             H
ATOM     12  H33  PLA    1      10.771  43.857  25.067  1.00  0.00             H
ATOM     13  N4   PLA    1  

In [13]:
!grep CHOL traj_POPC_wH.pdb | head -20

ATOM   2681  C1  CHOL   21       8.750  35.790  36.370  1.00  0.00             C
ATOM   2682  C2  CHOL   21       9.800  36.660  35.670  1.00  0.00             C
ATOM   2683  C3  CHOL   21       9.370  37.990  35.030  1.00  0.00             C
ATOM   2684  C4  CHOL   21       8.600  37.850  33.710  1.00  0.00             C
ATOM   2685  C5  CHOL   21       9.340  36.970  32.700  1.00  0.00             C
ATOM   2686  O6  CHOL   21       8.500  36.770  31.560  1.00  0.00             O
ATOM   2687  H   CHOL   21       8.170  37.670  31.260  1.00  0.00             H
ATOM   2688  C8  CHOL   21       9.890  35.630  33.200  1.00  0.00             C
ATOM   2689  C9  CHOL   21      10.480  35.810  34.600  1.00  0.00             C
ATOM   2690  C10 CHOL   21      11.690  35.180  34.860  1.00  0.00             C
ATOM   2691  C11 CHOL   21      12.450  35.230  36.190  1.00  0.00             C
ATOM   2692  C12 CHOL   21      11.660  35.960  37.280  1.00  0.00             C
ATOM   2693  C13 CHOL   21  

Yeah that is fine! POPC have all hydrogens reconstructed, while cholesterol does not (we can only see the polar hydrogen on the OH group which is natively present in a united-atom structure).

## Dealing with cholesterol

Now that POPC have all hydrogens, we can restart from that pdb file to reconstruct the hydrogens of cholesterol. Thus we can use the flags `-c` with the pdb file we just generated `traj_POPC_wH.pdb` (Hs on POPC but not POPE). The `-t` will be used with the same trajectory with H on POPC (but not POPE!) `traj_POPC_wH.xtc`. We output order parameters for POPE with `-o`. Last, `-opx` will produce a file with hydrogens on POPE.

In [14]:
!buildH -c traj_POPC_wH.pdb -t traj_POPC_wH.xtc -l Berger_CHOL -d Berger_CHOL.def -o OP_CHOL.out -opx traj_POPC_CHOL_wH

Constructing the system...
/home/pierre/.soft/miniconda3/envs/buildh/lib/python3.9/site-packages/MDAnalysis/topology/base.py:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  residx = np.zeros_like(criteria[0], dtype=np.int)
/home/pierre/.soft/miniconda3/envs/buildh/lib/python3.9/site-packages/MDAnalysis/coordinates/XDR.py:216: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn("Reload offsets from trajectory\n "
/home/pierre/.soft/miniconda3/envs/buildh/lib/python3.9/site-packages/MDAnalysis/lib/

In [15]:
!ls

Berger_CHOL.def
Berger_POPC.def
Berger_POPC.def.1
endCONF_OK.gro
img
Notebook_01_buildH_calc_OP.ipynb
Notebook_02_buildH_calc_OP_outputwH.ipynb
Notebook_03_mixture_POPC_POPE.ipynb
Notebook_04_library.ipynb
Notebook_05_mixture_POPC_cholesterol.ipynb
OP_CHOL.out
OP_POPC.dat
OP_POPC.out
popc0-25ns_dt1000.xtc
popcCHOL34molPER0-25ns_OK_dt1000.xtc
start_128popc.pdb
traj_POPC_CHOL_wH.pdb
traj_POPC_CHOL_wH.xtc
traj_POPC_wH.pdb
traj_POPC_wH.xtc


Great, we can now look at the order parameters of cholesterol:

In [16]:
!head OP_CHOL.out

# OP_name            resname atom1 atom2  OP_mean OP_stddev OP_stem
#--------------------------------------------------------------------
cholesterol5a        CHOL    C5    H51   -0.36784  0.06484  0.00978
cholesterol8a        CHOL    C8    H81   -0.25563  0.13526  0.02039
cholesterol8b        CHOL    C8    H82   -0.31507  0.07072  0.01066
cholesterol10a       CHOL    C10   H101   0.01633  0.16780  0.02530
cholesterol11a       CHOL    C11   H111  -0.28548  0.08254  0.01244
cholesterol11b       CHOL    C11   H112  -0.40213  0.05298  0.00799
cholesterol12a       CHOL    C12   H121  -0.43874  0.04642  0.00700
cholesterol18a       CHOL    C18   H181  -0.31974  0.07137  0.01076


And check whether POPC **and** POPE have all hydrogens reconstructed:

In [17]:
!grep CHOL traj_POPC_CHOL_wH.pdb | head -20

ATOM   2681  C1  CHOL   21       8.750  35.790  36.370  1.00  0.00             C
ATOM   2682  H11 CHOL   21       8.249  36.375  37.141  1.00  0.00             H
ATOM   2683  H12 CHOL   21       9.238  34.929  36.827  1.00  0.00             H
ATOM   2684  H13 CHOL   21       8.017  35.447  35.640  1.00  0.00             H
ATOM   2685  C2  CHOL   21       9.800  36.660  35.670  1.00  0.00             C
ATOM   2686  C3  CHOL   21       9.370  37.990  35.030  1.00  0.00             C
ATOM   2687  H31 CHOL   21      10.267  38.580  34.839  1.00  0.00             H
ATOM   2688  H32 CHOL   21       8.733  38.517  35.740  1.00  0.00             H
ATOM   2689  C4  CHOL   21       8.600  37.850  33.710  1.00  0.00             C
ATOM   2690  H41 CHOL   21       7.627  37.405  33.917  1.00  0.00             H
ATOM   2691  H42 CHOL   21       8.462  38.841  33.277  1.00  0.00             H
ATOM   2692  C5  CHOL   21       9.340  36.970  32.700  1.00  0.00             C
ATOM   2693  H51 CHOL   21  

In [18]:
!grep PLA traj_POPC_CHOL_wH.pdb | head -20

ATOM      1  C1   PLA    1      10.670  42.050  27.850  1.00  0.00             C
ATOM      2  H11  PLA    1      10.730  41.120  27.280  1.00  0.00             H
ATOM      3  H12  PLA    1       9.960  41.930  28.670  1.00  0.00             H
ATOM      4  H13  PLA    1      11.650  42.290  28.250  1.00  0.00             H
ATOM      5  C2   PLA    1       8.900  42.780  26.440  1.00  0.00             C
ATOM      6  H21  PLA    1       8.970  41.830  25.920  1.00  0.00             H
ATOM      7  H22  PLA    1       8.560  43.550  25.750  1.00  0.00             H
ATOM      8  H23  PLA    1       8.190  42.690  27.260  1.00  0.00             H
ATOM      9  C3   PLA    1      11.120  43.130  25.800  1.00  0.00             C
ATOM     10  H31  PLA    1      11.120  42.140  25.350  1.00  0.00             H
ATOM     11  H32  PLA    1      12.130  43.390  26.120  1.00  0.00             H
ATOM     12  H33  PLA    1      10.770  43.860  25.070  1.00  0.00             H
ATOM     13  N4   PLA    1  

## Conclusion

In this notebook, we showed you how to use **buildH** on a mixture of POPC/cholesterol lipids. **buildH** has to be launched for each lipid separately. If a trajectory with all hydrogens on all lipids is wanted, one has to produce intermediate trajectories at each step. In the figure below is shown a snapshot with all the reconstructed hydrogens on cholesterol molecules (POPC and water are not shown for clarity, image rendered with [VMD](https://www.ks.uiuc.edu/Research/vmd/)).

![H reconstruction on a POPC/POPE mixture with buildH](img/cholesterols.png)